# Making a Geodataframe from Fire location data

In [ ]:
#Import the necessary Python moduless
import pandas as pd
import geopandas as gpd
import numpy as np
from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster
from folium import IFrame
import shapely
from shapely.geometry import Point
import unicodedata
import pysal as ps

In [ ]:
dba = pd.read_csv('../data/vector/fire_archive_V1_2871.csv')

def lookup(s):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    return s.map({date:pd.to_datetime(date) for date in s.unique()})

dba.loc[:,'date']=lookup(dba['acq_date'])

dba['monthday'] = dba['date'].dt.strftime('%j')
dba['fileformate'] = dba['date'].dt.strftime('%Y_%m_%d')
dba['year'] = dba['date'].dt.strftime('%Y')


yearlist=dba.drop_duplicates('year')
yearlist1=yearlist['year'].tolist()

dba

In [ ]:
#First create a GeoSeries of crime locations by converting coordinates to Shapely geometry objects
#Specify the coordinate system ESPG4326 which represents the standard WGS84 coordinate system
fire_geo = gpd.GeoSeries(dba.apply(lambda z: Point(z['longitude'], z['latitude']), 1),crs={'init': 'epsg:4326'})

#Create a geodataframe from the pandas dataframe and the geoseries of shapely geometry objects
firepts = gpd.GeoDataFrame(dba.drop(['longitude', 'latitude'], 1), geometry=fire_geo)
print(firepts.head())

# calculate state level density of fire observation

In [ ]:
#Read tracts shapefile into GeoDataFrame
state_poly = gpd.read_file('India_states.shp')

#Generate Counts of Assaults per Census Tract
#Spatially join census tracts to assaults (after projecting) and then group by Tract FIPS while counting the number of crimes
point_counts = gpd.tools.sjoin(firepts.to_crs(state_poly.crs), state_poly.reset_index()).groupby('censuscode').size()

#Calculate Assault Density, converting square meters to square miles.
#state_poly['firePSqm'] = (point_counts/(state_poly.geometry.area)).fillna(0)
#state_poly = state_poly.reset_index()
#print(state_poly.head())
point_counts=pd.DataFrame(point_counts)
point_counts['censuscode']=point_counts.index
point_counts.columns=['fireCountPSqm','censuscode']

points_poly=pd.merge(state_poly,point_counts,on='censuscode')
points_poly

# Adding chloropleth map layer

In [ ]:
#Create SF basemap specifying map center, zoom level, and using the default OpenStreetMap tiles
fire_map = folium.Map([19.17, 72.98], zoom_start = 12)

def add_choropleth(mapobj, gdf, id_field, value_field, fill_color = 'YlOrRd', fill_opacity = 0.6, 
                    line_opacity = 0.2, num_classes = 5, classifier = 'Fisher_Jenks'):
    #Allow for 3 Pysal map classifiers to display data
    #Generate list of breakpoints using specified classification scheme. List of breakpoint will be input to choropleth function
    if classifier == 'Fisher_Jenks':
        threshold_scale=ps.esda.mapclassify.Fisher_Jenks(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Equal_Interval':
        threshold_scale=ps.esda.mapclassify.Equal_Interval(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Quantiles':
        threshold_scale=ps.esda.mapclassify.Quantiles(gdf[value_field], k = num_classes).bins.tolist()
    
    #Convert the GeoDataFrame to WGS84 coordinate reference system
    gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
    
    #Call Folium choropleth function, specifying the geometry as a the WGS84 dataframe converted to GeoJSON, the data as 
    #the GeoDataFrame, the columns as the user-specified id field and and value field.
    #key_on field refers to the id field within the GeoJSON string
    mapobj.choropleth(gdf_wgs84.to_json(), data = gdf,
                columns = [id_field, value_field], key_on = 'feature.properties.{}'.format(id_field),
                fill_color = fill_color, fill_opacity = fill_opacity, line_opacity = line_opacity,  
                threshold_scale = threshold_scale)
    return mapobj

#Update basemap with choropleth
fire_map =add_choropleth(fire_map,points_poly, 'censuscode','fireCountPSqm')

# Adding point layer

In [ ]:
def add_point_clusters(mapobj, gdf, popup_field_list):
    #Create empty lists to contain the point coordinates and the point pop-up information
    coords, popups = [], [] 
    #Loop through each record in the GeoDataFrame
    for i, row in gdf.iterrows():
        #Append lat and long coordinates to "coords" list
        coords.append([row.geometry.y, row.geometry.x])
        #Create a string of HTML code used in the IFrame popup
        #Join together the fields in "popup_field_list" with a linebreak between them
        label = '<br>'.join([row[field] for field in popup_field_list])
        #Append an IFrame that uses the HTML string to the "popups" list 
        popups.append(IFrame(label, width = 300, height = 100))
        
    #Create a Folium feature group for this layer, since we will be displaying multiple layers
    pt_lyr = folium.FeatureGroup(name = 'pt_lyr')
    
    #Add the clustered points of crime locations and popups to this layer
    pt_lyr.add_children(MarkerCluster(locations = coords, popups = popups))
    
    #Add this point layer to the map object
    mapobj.add_children(pt_lyr)
    return mapobj

#Update choropleth with point clusters
fire_map = add_point_clusters(fire_map,firepts , ['instrument'])

In [ ]:
folium.LayerControl().add_to(fire_map) #Add layer control to toggle on/off
#crime_map.save('sf_assaults.html') #save HTML
fire_map #display map